In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np, os
import matplotlib.pyplot as plt

from pathlib import Path
import yaml
import re
import datetime

import pyarrow as pa
import pyarrow.parquet as pq

import gbd_mapping as gm
from vivarium import Artifact

from db_queries import get_ids, get_outputs, get_population, get_covariate_estimates
from get_draws.api import get_draws

import vivarium_helpers as vh
import vivarium_helpers.id_helper as idh
from vivarium_helpers.vph_output.operations import VPHOperator
from vivarium_helpers.vph_output.measures import VPHResults
from vivarium_helpers.utils import convert_to_categorical, constant_categorical, print_memory_usage, Timer
from vivarium_helpers.vph_output import cleaning
from vivarium_helpers.vph_output.loading import load_draws_from_keyspace_files
from vivarium_helpers.projects.alzheimers import loading, population
from vivarium_helpers.projects.alzheimers.population import RunType
from vivarium_helpers.vph_artifact.operations import convert_to_sim_format
from vivarium_helpers.projects.alzheimers.results import AlzheimersResultsProcessor

!date
!whoami
!pwd

Tue Nov 18 12:29:10 PST 2025
ndbs
/mnt/share/code/ndbs/vivarium_research_alzheimers/results_tables


In [3]:
# from vivarium_helpers.projects.alzheimers.results import AlzheimersResultsProcessor

# Define data directories

In [4]:
# Project directory
project_dir = Path('/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/')

# Output directory to store final results for client
output_dir = Path(r"J:\Project\simulation_science\alzheimers\results_in_progress".replace('\\', '/').replace('J:', '/snfs1'))
print(output_dir.exists())

# Output from multistate life table
mslt_output_dir = output_dir.parent / 'results_11_14_2025_mslt'
print(mslt_output_dir.exists())
output_dir

True
True


PosixPath('/snfs1/Project/simulation_science/alzheimers/results_in_progress')

In [5]:
# For testing: Run directory containing model 8.3 results for all
# locations
model_run_subdir = 'results/abie_consistent_model_test/united_states_of_america/2025_10_28_08_55_05/'

# Results directory for model 8.3, for testing
results_dirs = project_dir / model_run_subdir / 'results/'

# Artifact for models 8.3 - 8.7
artifact_model_number = '8.3'


In [6]:
# Model 8.4 results (final runs for 10/31/25 intermediate results)
# Each batch run contains all locations, 100 random seeds, and 3 or 4 draws
model_run_dir_8_4 = Path('/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/results/model8.4/model_spec')

# Model 8.7 results (updated final run completed on 11/10/2025)
model_run_dir_8_7 = Path('/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/results/model8.7/model_spec')

# Batch runs are stored in "timestamp" subdirectories of the model run
# directory
with os.scandir(model_run_dir_8_7) as entries:
    batch_run_dirs_8_7 = [entry.path for entry in entries]

# Select a model and filter for testing
batch_run_dirs = batch_run_dirs_8_7[:2]

batch_results_dirs = []
for run_dir in batch_run_dirs:
    if run_dir.endswith('2025_10_29_20_41_39'):
        # One batch had to be deduplicated
        results_dir = run_dir + '/deduplicated_results'
    else:
        results_dir = run_dir + '/results'
    batch_results_dirs.append(results_dir)
batch_results_dirs

batch_results_dirs


['/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/results/model8.7/model_spec/2025_11_05_15_36_29/results',
 '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/results/model8.7/model_spec/2025_11_06_06_42_37/results']

In [16]:
results = AlzheimersResultsProcessor('8.3', RunType.FINAL, batch_run_dirs=batch_run_dirs)

In [15]:
results.draws

[457, 169, 323, 258, 446]

# Define locations, column datatypes, and directory maps

In [7]:
locations = [
    'United States of America',
    'Brazil',
    'China',
    'Germany',
    'Israel',
    'Japan',
    'Spain',
    'Sweden',
    'Taiwan (Province of China)',
    'United Kingdom',
]

colname_to_dtype = loading.get_column_dtypes(locations)

location_to_results_dir = loading.get_location_results_dict(results_dirs)

location_to_artifact_path = loading.get_location_artifact_dict(
    locations, artifact_model_number)

# This is needed to assign locations in the model results
location_to_artifact_path

{'United States of America': '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model8.3/united_states_of_america.hdf',
 'Brazil': '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model8.3/brazil.hdf',
 'China': '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model8.3/china.hdf',
 'Germany': '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model8.3/germany.hdf',
 'Israel': '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model8.3/israel.hdf',
 'Japan': '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model8.3/japan.hdf',
 'Spain': '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model8.3/spain.hdf',
 'Sweden': '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model8.3/sweden.hdf',
 'Taiwan (Province of China)': '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifact

# Load one artifact to define age bins

### And read draws from `keyspace.yaml` files to filter Artifact data

In [8]:
draws = load_draws_from_keyspace_files(batch_run_dirs)
draws

[457, 169, 323, 258, 446]

In [9]:
artifact_path = location_to_artifact_path[locations[0]]
print(artifact_path)
age_map = loading.get_age_map(artifact_path, age_group_dtype=colname_to_dtype['age_group'])
age_map

/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model8.3/united_states_of_america.hdf


,age_group_id,age_group_name,age_start,age_end,age_group
0,10,25 to 29,25.0,30.0,25_to_29
1,11,30 to 34,30.0,35.0,30_to_34
2,12,35 to 39,35.0,40.0,35_to_39
3,13,40 to 44,40.0,45.0,40_to_44
4,14,45 to 49,45.0,50.0,45_to_49
5,15,50 to 54,50.0,55.0,50_to_54
6,16,55 to 59,55.0,60.0,55_to_59
7,17,60 to 64,60.0,65.0,60_to_64
8,18,65 to 69,65.0,70.0,65_to_69
9,19,70 to 74,70.0,75.0,70_to_74


# Test loading data

In [10]:
# Stats for reading deaths.parquet with no extra filters:
# 28 seconds to load 2 batches with 1 location group, 20 MB after concatenation,
# max 1877 MB before aggregating seeds
# 31 seconds to load 2 batches with 3 location groups, 86 MB after concatenation,
# max 751 MB before aggregating seeds
with Timer():
    test = loading.load_measure_from_batch_runs(
        'deaths', batch_results_dirs, locations, n_location_groups=1,
        artifact_model_number=artifact_model_number,
        colname_to_dtype=colname_to_dtype,
    )
test.dtypes

[('event_year', '>=', '2025')]
/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/results/model8.7/model_spec/2025_11_05_15_36_29/results
{'United States of America': '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model8.3/united_states_of_america.hdf', 'Brazil': '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model8.3/brazil.hdf', 'China': '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model8.3/china.hdf', 'Germany': '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model8.3/germany.hdf', 'Israel': '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model8.3/israel.hdf', 'Japan': '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model8.3/japan.hdf', 'Spain': '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model8.3/spain.hdf', 'Sweden': '/mnt/team/simulation_science/pub/models/vivarium_csu_alz

age_group        category
artifact_path    category
entity           category
entity_type      category
event_year          int16
input_draw          int16
measure          category
scenario         category
sex              category
value             float64
location         category
dtype: object

# Load population structure and initial all-states prevlaence from Artifact

In [11]:
art_pop_structure = loading.load_artifact_data(
    'population.structure', None, location_to_artifact_path)
art_pop_structure.tail()

draw_0  \
location       sex  age_start age_end year_start year_end                  
United Kingdom Male 95.0      125.0   2046       2047      129365.649457   
                                      2047       2048      129176.872428   
                                      2048       2049      128215.394230   
                                      2049       2050      127765.330292   
                                      2050       2051      128213.393685   

                                                                  draw_1  \
location       sex  age_start age_end year_start year_end                  
United Kingdom Male 95.0      125.0   2046       2047      130775.330848   
                                      2047       2048      131427.429524   
                                      2048       2049      131705.587524   
                                      2049       2050      131992.359105   
                                      2050       2051      133220.130675   

                                                                  draw_2  \
location       sex  age_start age_end year_start year_end                  
United Kingdom Male 95.0      125.0   2046       2047      132470.262731   
                                      2047       2048      133135.096509   
                                      2048       2049      132988.854403   
                                      2049       2050      132745.174212   
                                      2050       2051      133421.143899   

                                                                  draw_3  \
location       sex  age_start age_end year_start year_end                  
United Kingdom Male 95.0      125.0   2046       2047      122141.117458   
                                      2047       2048      122206.362659   
                                      2048       2049      121848.151327   
                                      2049       2050      121566.569294   
                                      2050       2051      122248.029681   

                                                                  draw_4  \
location       sex  age_start age_end year_start year_end                  
United Kingdom Male 95.0      125.0   2046       2047      122669.816649   
                                      2047       2048      121990.475534   
                                      2048       2049      120928.626569   
                                      2049       2050      120010.799977   
                                      2050       2051      119754.120437   

                                                                  draw_5  \
location       sex  age_start age_end year_start year_end                  
United Kingdom Male 95.0      125.0   2046       2047      116553.291993   
                                      2047       2048      116216.514579   
                                      2048       2049      115242.163772   
                                      2049       2050      114127.489703   
                                      2050       2051      113654.996368   

                                                                  draw_6  \
location       sex  age_start age_end year_start year_end                  
United Kingdom Male 95.0      125.0   2046       2047      137240.957664   
                                      2047       2048      137226.419110   
                                      2048       2049      136633.081048   
                                      2049       2050      136240.266101   
                                      2050       2051      136919.776385   

                                                                  draw_7  \
location       sex  age_start age_end year_start year_end                  
United Kingdom Male 95.0      125.0   2046       2047      146303.198839   
                                      2047       2048      147381.840121   
                               

In [12]:
art_all_states_initial_prev = loading.load_artifact_data(
    'population.scaling_factor', None, location_to_artifact_path)
art_all_states_initial_prev.tail()

draw_0  \
location       sex    age_start age_end year_start year_end             
United Kingdom Female 80        85      2025       2026      0.184057   
                      85        90      2025       2026      0.249261   
                      90        95      2025       2026      0.288238   
                      95        100     2025       2026      0.262417   
                      100       105     2025       2026      0.253172   

                                                               draw_1  \
location       sex    age_start age_end year_start year_end             
United Kingdom Female 80        85      2025       2026      0.205264   
                      85        90      2025       2026      0.251477   
                      90        95      2025       2026      0.285702   
                      95        100     2025       2026      0.263717   
                      100       105     2025       2026      0.257814   

                                                               draw_2  \
location       sex    age_start age_end year_start year_end             
United Kingdom Female 80        85      2025       2026      0.201615   
                      85        90      2025       2026      0.250982   
                      90        95      2025       2026      0.254878   
                      95        100     2025       2026      0.252117   
                      100       105     2025       2026      0.248932   

                                                               draw_3  \
location       sex    age_start age_end year_start year_end             
United Kingdom Female 80        85      2025       2026      0.203185   
                      85        90      2025       2026      0.279132   
                      90        95      2025       2026      0.292849   
                      95        100     2025       2026      0.276680   
                      100       105     2025       2026      0.272355   

                                                               draw_4  \
location       sex    age_start age_end year_start year_end             
United Kingdom Female 80        85      2025       2026      0.178318   
                      85        90      2025       2026      0.262662   
                      90        95      2025       2026      0.297609   
                      95        100     2025       2026      0.262065   
                      100       105     2025       2026      0.252686   

                                                               draw_5  \
location       sex    age_start age_end year_start year_end             
United Kingdom Female 80        85      2025       2026      0.175798   
                      85        90      2025       2026      0.236341   
                      90        95      2025       2026      0.266154   
                      95        100     2025       2026      0.260805   
                      100       105     2025       2026      0.253852   

                                                               draw_6  \
location       sex    age_start age_end year_start year_end             
United Kingdom Female 80        85      2025       2026      0.165233   
                      85        90      2025       2026      0.308766   
                      90        95      2025       2026      0.328095   
                      95        100     2025       2026      0.268706   
                      100       105     2025       2026      0.252204   

                                                               draw_7  \
location       sex    age_start age_end year_start year_end             
United Kingdom Female 80        85      2025       2026      0.152835   
                      85        90      2025       2026      0.292557   
                      90        95      2025       2026      0.294896   
                      95        100     2025       2026      0.252330   
                      100       105     2

# Calculate initial real-world prevalence counts and model scale

In [13]:
art_all_states_initial_prev_counts = population.get_initial_real_world_population(
    art_pop_structure, art_all_states_initial_prev
)
art_all_states_initial_prev_counts.tail()

draw_0  \
location                 sex  age_start age_end year_start year_end                  
United States of America Male 75.0      80.0    2022       2023      279503.716159   
                              80.0      85.0    2022       2023      295491.963153   
                              85.0      90.0    2022       2023      265660.836082   
                              90.0      95.0    2022       2023      135009.931912   
                              95.0      100.0   2022       2023       29288.078689   

                                                                            draw_1  \
location                 sex  age_start age_end year_start year_end                  
United States of America Male 75.0      80.0    2022       2023      241421.537335   
                              80.0      85.0    2022       2023      334542.816129   
                              85.0      90.0    2022       2023      279333.123360   
                              90.0      95.0    2022       2023      140544.500518   
                              95.0      100.0   2022       2023       30698.389085   

                                                                            draw_2  \
location                 sex  age_start age_end year_start year_end                  
United States of America Male 75.0      80.0    2022       2023      303005.456325   
                              80.0      85.0    2022       2023      352919.503418   
                              85.0      90.0    2022       2023      259372.093530   
                              90.0      95.0    2022       2023      117802.312175   
                              95.0      100.0   2022       2023       30107.564684   

                                                                            draw_3  \
location                 sex  age_start age_end year_start year_end                  
United States of America Male 75.0      80.0    2022       2023      245046.790482   
                              80.0      85.0    2022       2023      327523.755730   
                              85.0      90.0    2022       2023      273638.950137   
                              90.0      95.0    2022       2023      135061.861965   
                              95.0      100.0   2022       2023       32246.260410   

                                                                            draw_4  \
location                 sex  age_start age_end year_start year_end                  
United States of America Male 75.0      80.0    2022       2023      291032.019730   
                              80.0      85.0    2022       2023      309471.027115   
                              85.0      90.0    2022       2023      256249.409965   
                              90.0      95.0    2022       2023      138174.629778   
                              95.0      100.0   2022       2023       30766.214728   

                                                                            draw_5  \
location                 sex  age_start age_end year_start year_end                  
United States of America Male 75.0      80.0    2022       2023      292173.888069   
                              80.0      85.0    2022       2023      296260.437344   
                              85.0      90.0    2022       2023      220125.858697   
                              90.0      95.0    2022       2023      113243.661152   
                              95.0      100.0   2022       2023       29113.490401   

                                                                            draw_6  \
location                 sex  age_start age_end year_start year_end                  
United States of America Male 75.0      80.0    2022       2023      319151.434213   
                              80.0      85.0    2022       2023      265026.502337   
                              85.0      90.0    2022       2023      295555.094216   
                            

In [14]:
# NOTE: Make sure initial sim population is accurate!
initial_simulation_population = population.get_initial_simulation_population(
    RunType.FINAL)

print(f'{initial_simulation_population=}')

art_model_scale = population.calculate_model_scale(
    initial_simulation_population,
    art_all_states_initial_prev_counts,
)
art_model_scale

initial_simulation_population=2000000


,draw_0,draw_1,draw_2,draw_3,draw_4,draw_5,draw_6,draw_7,draw_8,draw_9,...,draw_490,draw_491,draw_492,draw_493,draw_494,draw_495,draw_496,draw_497,draw_498,draw_499
location,,,,,,,,,,,,,,,,,,,,,
Brazil,1.182478,1.062553,1.144685,1.078614,1.082753,1.121963,1.003505,1.068713,1.068239,1.220286,...,1.290358,1.060123,1.086799,1.233531,0.984699,1.172564,1.172570,1.039336,1.177133,1.124321
China,0.133424,0.130108,0.114296,0.115621,0.123448,0.119143,0.117581,0.124027,0.115181,0.126465,...,0.119680,0.125693,0.127735,0.134309,0.126757,0.123859,0.124902,0.119387,0.127926,0.123437
Germany,0.863308,0.892693,0.882412,0.879660,0.779863,0.862006,0.874659,0.883173,0.857328,0.946077,...,0.948708,0.848518,0.866878,0.955230,0.949687,0.931259,0.830667,0.891217,0.877104,0.868272
Israel,41.254628,40.310785,40.210343,40.057537,39.166235,44.012209,35.797113,38.209922,41.226551,40.837404,...,41.809532,40.213923,40.180520,45.482468,47.228409,38.099635,38.145320,34.621104,48.494263,40.043924
Japan,0.586866,0.574811,0.618108,0.553387,0.564055,0.540442,0.488238,0.511745,0.570487,0.564432,...,0.536199,0.591534,0.595133,0.650132,0.621924,0.603336,0.566684,0.576934,0.585316,0.567870
Spain,3.211223,3.418895,3.073433,2.871964,3.086059,3.235590,2.882333,2.904501,3.250294,2.987198,...,3.452614,3.070298,3.679101,3.409524,3.012754,3.228092,2.819038,2.934665,3.112781,3.208168
Sweden,13.614881,13.790971,11.796806,11.934329,12.544275,12.395964,11.855079,13.491354,13.650575,12.357657,...,13.678809,12.871456,13.393983,14.436751,12.375910,11.803971,12.878710,12.507280,14.370234,12.398862
Taiwan (Province of China),8.808232,7.720413,9.725660,8.515740,8.443536,8.972456,8.809221,9.256961,7.936908,8.524695,...,8.314534,8.305947,8.938951,9.810819,9.125070,8.416122,8.602790,8.931823,8.469918,8.560164
United Kingdom,2.099057,2.036624,2.077655,2.012311,2.061171,2.033260,2.023333,2.027407,1.980920,1.990865,...,1.905267,1.952344,2.208905,2.390574,2.156457,2.099913,1.927622,2.024944,2.021172,2.005293


# Reformat model scale and population structure to sim format

In [15]:
model_scale_sim = convert_to_sim_format(art_model_scale, draws, None, age_map, colname_to_dtype)
model_scale_sim

,location,input_draw,value
0,Brazil,169,1.053388
1,Brazil,258,0.943491
2,Brazil,323,1.059915
3,Brazil,446,0.982799
4,Brazil,457,1.151080
5,China,169,0.116177
6,China,258,0.106308
7,China,323,0.112346
8,China,446,0.120095
9,China,457,0.125744


In [27]:
f'Must pass one of {", ".join(str(x) for x in RunType)} for run_type'

'Must pass one of RunType.V_AND_V, RunType.FINAL for run_type'

In [23]:
",".join([str(x) for x in RunType])

'RunType.V_AND_V,RunType.FINAL'

In [26]:
population.get_initial_simulation_population(4)

ValueError: Must pass one of RunType.V_AND_V, RunType.FINAL for run_type

# Create a results object and load population data

In [17]:
results = AlzheimersResultsProcessor('8.3', RunType.FINAL, batch_run_dirs=batch_run_dirs)

In [18]:
results.load_population_data()
results.model_scale

,location,input_draw,value
0,Brazil,169,1.053388
1,Brazil,258,0.943491
2,Brazil,323,1.059915
3,Brazil,446,0.982799
4,Brazil,457,1.151080
5,China,169,0.116177
6,China,258,0.106308
7,China,323,0.112346
8,China,446,0.120095
9,China,457,0.125744


In [19]:
results.person_time

,location,sex,event_year,input_draw,value,age_group
0,United States of America,Female,2025,169,1.156020e+07,25_to_29
1,United States of America,Female,2025,258,1.094300e+07,25_to_29
2,United States of America,Female,2025,323,1.051324e+07,25_to_29
3,United States of America,Female,2025,446,1.097049e+07,25_to_29
4,United States of America,Female,2025,457,1.121058e+07,25_to_29
...,...,...,...,...,...,...
113995,United Kingdom,Male,2100,169,1.309761e+05,95_plus
113996,United Kingdom,Male,2100,258,1.554175e+05,95_plus
113997,United Kingdom,Male,2100,323,1.205289e+05,95_plus
113998,United Kingdom,Male,2100,446,1.254920e+05,95_plus


In [32]:
with Timer():
    results.load_measure_from_batch_runs('deaths', filters=loading.FINAL_RESULTS_FILTERS['deaths'])

[('event_year', '>=', '2025'), ('entity', '=', 'alzheimers_disease_state')]
{'all': PosixPath('/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/results/model8.7/model_spec/2025_11_05_15_36_29/results')}
{'United States of America': '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model8.3/united_states_of_america.hdf', 'Brazil': '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model8.3/brazil.hdf', 'China': '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model8.3/china.hdf', 'Germany': '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model8.3/germany.hdf', 'Israel': '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model8.3/israel.hdf', 'Japan': '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model8.3/japan.hdf', 'Spain': '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model8.3/spain.hdf', 

In [33]:
results.data.deaths

,age_group,artifact_path,entity,entity_type,event_year,input_draw,measure,scenario,sex,value,location
0,25_to_29,/mnt/team/simulation_science/pub/models/vivari...,alzheimers_disease_state,cause,2025,169,deaths,baseline,Female,0.0,Japan
1,25_to_29,/mnt/team/simulation_science/pub/models/vivari...,alzheimers_disease_state,cause,2025,169,deaths,baseline,Male,0.0,Japan
2,25_to_29,/mnt/team/simulation_science/pub/models/vivari...,alzheimers_disease_state,cause,2025,169,deaths,bbbm_testing,Female,0.0,Japan
3,25_to_29,/mnt/team/simulation_science/pub/models/vivari...,alzheimers_disease_state,cause,2025,169,deaths,bbbm_testing,Male,0.0,Japan
4,25_to_29,/mnt/team/simulation_science/pub/models/vivari...,alzheimers_disease_state,cause,2025,169,deaths,bbbm_testing_and_treatment,Female,0.0,Japan
...,...,...,...,...,...,...,...,...,...,...,...
341995,95_plus,/mnt/team/simulation_science/pub/models/vivari...,alzheimers_disease_state,cause,2100,446,deaths,baseline,Male,6114.0,Brazil
341996,95_plus,/mnt/team/simulation_science/pub/models/vivari...,alzheimers_disease_state,cause,2100,446,deaths,bbbm_testing,Female,17529.0,Brazil
341997,95_plus,/mnt/team/simulation_science/pub/models/vivari...,alzheimers_disease_state,cause,2100,446,deaths,bbbm_testing,Male,6114.0,Brazil
341998,95_plus,/mnt/team/simulation_science/pub/models/vivari...,alzheimers_disease_state,cause,2100,446,deaths,bbbm_testing_and_treatment,Female,17810.0,Brazil


In [39]:
342000 /3 * (2+3)

570000.0

In [35]:
deaths_prepped = results.process_deaths(results.data.deaths)
deaths_prepped

9.581504 MB measure
3.197504 MB minuend
6.389504 MB subtrahend
2.060987 MB minuend re-indexed
4.112987 MB subtrahend re-indexed
4.113047 MB difference
7.301636 MB difference with reset index
7.529961 MB final difference


,age_group,artifact_path,entity,entity_type,event_year,input_draw,measure,scenario,sex,value,location,metric
0,25_to_29,/mnt/team/simulation_science/pub/models/vivari...,alzheimers_disease_state,cause,2025,169,Deaths Associated with AD,baseline,Female,0.0,Japan,Number
1,25_to_29,/mnt/team/simulation_science/pub/models/vivari...,alzheimers_disease_state,cause,2025,169,Deaths Associated with AD,baseline,Male,0.0,Japan,Number
2,25_to_29,/mnt/team/simulation_science/pub/models/vivari...,alzheimers_disease_state,cause,2025,169,Deaths Associated with AD,bbbm_testing,Female,0.0,Japan,Number
3,25_to_29,/mnt/team/simulation_science/pub/models/vivari...,alzheimers_disease_state,cause,2025,169,Deaths Associated with AD,bbbm_testing,Male,0.0,Japan,Number
4,25_to_29,/mnt/team/simulation_science/pub/models/vivari...,alzheimers_disease_state,cause,2025,169,Deaths Associated with AD,bbbm_testing_and_treatment,Female,0.0,Japan,Number
...,...,...,...,...,...,...,...,...,...,...,...,...
569995,95_plus,/mnt/team/simulation_science/pub/models/vivari...,alzheimers_disease_state,cause,2100,258,Averted Deaths Associated with AD,bbbm_testing_and_treatment,Male,-277.0,Brazil,Number
569996,95_plus,/mnt/team/simulation_science/pub/models/vivari...,alzheimers_disease_state,cause,2100,446,Averted Deaths Associated with AD,bbbm_testing,Female,0.0,Brazil,Number
569997,95_plus,/mnt/team/simulation_science/pub/models/vivari...,alzheimers_disease_state,cause,2100,446,Averted Deaths Associated with AD,bbbm_testing,Male,0.0,Brazil,Number
569998,95_plus,/mnt/team/simulation_science/pub/models/vivari...,alzheimers_disease_state,cause,2100,446,Averted Deaths Associated with AD,bbbm_testing_and_treatment,Female,-281.0,Brazil,Number


In [37]:
results.calculate_rate(deaths_prepped)

,age_group,event_year,input_draw,location,sex,artifact_path,entity,entity_type,measure,metric,scenario,value
0,25_to_29,2025,169,Brazil,Female,/mnt/team/simulation_science/pub/models/vivari...,alzheimers_disease_state,cause,Deaths Associated with AD,Number,baseline,0.000000
1,25_to_29,2025,169,Brazil,Female,/mnt/team/simulation_science/pub/models/vivari...,alzheimers_disease_state,cause,Deaths Associated with AD,Number,bbbm_testing,0.000000
2,25_to_29,2025,169,Brazil,Female,/mnt/team/simulation_science/pub/models/vivari...,alzheimers_disease_state,cause,Deaths Associated with AD,Number,bbbm_testing_and_treatment,0.000000
3,25_to_29,2025,169,Brazil,Female,/mnt/team/simulation_science/pub/models/vivari...,alzheimers_disease_state,cause,Averted Deaths Associated with AD,Number,bbbm_testing,0.000000
4,25_to_29,2025,169,Brazil,Female,/mnt/team/simulation_science/pub/models/vivari...,alzheimers_disease_state,cause,Averted Deaths Associated with AD,Number,bbbm_testing_and_treatment,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
569995,95_plus,2100,457,United States of America,Male,/mnt/team/simulation_science/pub/models/vivari...,alzheimers_disease_state,cause,Deaths Associated with AD,Number,baseline,0.004854
569996,95_plus,2100,457,United States of America,Male,/mnt/team/simulation_science/pub/models/vivari...,alzheimers_disease_state,cause,Deaths Associated with AD,Number,bbbm_testing,0.004854
569997,95_plus,2100,457,United States of America,Male,/mnt/team/simulation_science/pub/models/vivari...,alzheimers_disease_state,cause,Deaths Associated with AD,Number,bbbm_testing_and_treatment,0.004883
569998,95_plus,2100,457,United States of America,Male,/mnt/team/simulation_science/pub/models/vivari...,alzheimers_disease_state,cause,Averted Deaths Associated with AD,Number,bbbm_testing,0.000000


In [31]:
results.ops.compare_values(_25, results.data.deaths)

age_group,artifact_path,entity,entity_type,event_year,input_draw,location,measure,scenario,sex


# MSLT results

In [ ]:
!ls $mslt_output_dir

2025_11_14__14_11_42_outside_sim_results_draw_level.csv


In [97]:
mslt_results = pd.read_csv(mslt_output_dir / '2025_11_14__14_11_42_outside_sim_results_draw_level.csv', index_col=0)
mslt_results

,Year,Location,Age,Sex,Disease Stage,Scenario,Measure,Metric,Draw,Value
0,2025,Brazil,60_to_64,Female,Susceptible,BBBM Testing Only,BBBM Tests,Number,113,0.000000
1,2025,Brazil,60_to_64,Female,Susceptible,BBBM Testing Only,BBBM Tests,Number,13,0.000000
2,2025,Brazil,60_to_64,Female,Susceptible,BBBM Testing Only,BBBM Tests,Number,158,0.000000
3,2025,Brazil,60_to_64,Female,Susceptible,BBBM Testing Only,BBBM Tests,Number,169,0.000000
4,2025,Brazil,60_to_64,Female,Susceptible,BBBM Testing Only,BBBM Tests,Number,177,0.000000
...,...,...,...,...,...,...,...,...,...,...
151995,2100,United States of America,75_to_79,Male,Susceptible,BBBM Testing and Treatment,Improper Medication Uses,Number,46,41101.303607
151996,2100,United States of America,75_to_79,Male,Susceptible,BBBM Testing and Treatment,Improper Medication Uses,Number,460,41691.942867
151997,2100,United States of America,75_to_79,Male,Susceptible,BBBM Testing and Treatment,Improper Medication Uses,Number,480,39195.330257
151998,2100,United States of America,75_to_79,Male,Susceptible,BBBM Testing and Treatment,Improper Medication Uses,Number,499,40424.816911


In [114]:
mslt_results.Age.unique()

array(['60_to_64', '65_to_69', '70_to_74', '75_to_79'], dtype=object)

In [98]:
mslt_results_prepped = results.process_mslt_results(mslt_results)
mslt_results_prepped

,event_year,location,age_group,sex,disease_stage,scenario,measure,metric,input_draw,value
3,2025,Brazil,60_to_64,Female,Susceptible,BBBM Testing Only,BBBM Tests,Number,169,0.000000
8,2025,Brazil,60_to_64,Female,Susceptible,BBBM Testing Only,BBBM Tests,Number,258,0.000000
13,2025,Brazil,60_to_64,Female,Susceptible,BBBM Testing Only,BBBM Tests,Number,323,0.000000
18,2025,Brazil,60_to_64,Female,Susceptible,BBBM Testing Only,BBBM Tests,Number,446,0.000000
19,2025,Brazil,60_to_64,Female,Susceptible,BBBM Testing Only,BBBM Tests,Number,457,0.000000
...,...,...,...,...,...,...,...,...,...,...
151978,2100,United States of America,75_to_79,Male,Susceptible,BBBM Testing and Treatment,Medication Initiation,Number,169,42594.508978
151983,2100,United States of America,75_to_79,Male,Susceptible,BBBM Testing and Treatment,Medication Initiation,Number,258,40580.387735
151988,2100,United States of America,75_to_79,Male,Susceptible,BBBM Testing and Treatment,Medication Initiation,Number,323,38815.152311
151993,2100,United States of America,75_to_79,Male,Susceptible,BBBM Testing and Treatment,Medication Initiation,Number,446,40402.097304


# BBBM Test Counts

In [99]:
with Timer():
    bbbm_tests = results.load_measure_from_batch_runs(
        'counts_bbbm_tests', False, filters=loading.FINAL_RESULTS_FILTERS['counts_bbbm_tests'])

bbbm_tests

[('event_year', '>=', '2025'), ('bbbm_test_results', '!=', 'not_tested')]
{'all': PosixPath('/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/results/model8.7/model_spec/2025_11_05_15_36_29/results')}
{'United States of America': '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model8.3/united_states_of_america.hdf', 'Brazil': '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model8.3/brazil.hdf', 'China': '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model8.3/china.hdf', 'Germany': '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model8.3/germany.hdf', 'Israel': '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model8.3/israel.hdf', 'Japan': '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model8.3/japan.hdf', 'Spain': '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model8.3/spain.hdf', 'S

,age_group,artifact_path,bbbm_test_results,entity,entity_type,event_year,input_draw,measure,scenario,sex,value,location
0,25_to_29,/mnt/team/simulation_science/pub/models/vivari...,positive,bbbm_testing,testing,2025,169,counts_bbbm_tests,baseline,Female,0.0,Japan
1,25_to_29,/mnt/team/simulation_science/pub/models/vivari...,positive,bbbm_testing,testing,2025,169,counts_bbbm_tests,baseline,Male,0.0,Japan
2,25_to_29,/mnt/team/simulation_science/pub/models/vivari...,positive,bbbm_testing,testing,2025,169,counts_bbbm_tests,bbbm_testing,Female,0.0,Japan
3,25_to_29,/mnt/team/simulation_science/pub/models/vivari...,positive,bbbm_testing,testing,2025,169,counts_bbbm_tests,bbbm_testing,Male,0.0,Japan
4,25_to_29,/mnt/team/simulation_science/pub/models/vivari...,positive,bbbm_testing,testing,2025,169,counts_bbbm_tests,bbbm_testing_and_treatment,Female,0.0,Japan
...,...,...,...,...,...,...,...,...,...,...,...,...
683995,95_plus,/mnt/team/simulation_science/pub/models/vivari...,negative,bbbm_testing,testing,2100,446,counts_bbbm_tests,baseline,Male,0.0,Brazil
683996,95_plus,/mnt/team/simulation_science/pub/models/vivari...,negative,bbbm_testing,testing,2100,446,counts_bbbm_tests,bbbm_testing,Female,0.0,Brazil
683997,95_plus,/mnt/team/simulation_science/pub/models/vivari...,negative,bbbm_testing,testing,2100,446,counts_bbbm_tests,bbbm_testing,Male,0.0,Brazil
683998,95_plus,/mnt/team/simulation_science/pub/models/vivari...,negative,bbbm_testing,testing,2100,446,counts_bbbm_tests,bbbm_testing_and_treatment,Female,0.0,Brazil


In [100]:
bbbm_tests.query("bbbm_test_results=='positive'")

,age_group,artifact_path,bbbm_test_results,entity,entity_type,event_year,input_draw,measure,scenario,sex,value,location
0,25_to_29,/mnt/team/simulation_science/pub/models/vivari...,positive,bbbm_testing,testing,2025,169,counts_bbbm_tests,baseline,Female,0.0,Japan
1,25_to_29,/mnt/team/simulation_science/pub/models/vivari...,positive,bbbm_testing,testing,2025,169,counts_bbbm_tests,baseline,Male,0.0,Japan
2,25_to_29,/mnt/team/simulation_science/pub/models/vivari...,positive,bbbm_testing,testing,2025,169,counts_bbbm_tests,bbbm_testing,Female,0.0,Japan
3,25_to_29,/mnt/team/simulation_science/pub/models/vivari...,positive,bbbm_testing,testing,2025,169,counts_bbbm_tests,bbbm_testing,Male,0.0,Japan
4,25_to_29,/mnt/team/simulation_science/pub/models/vivari...,positive,bbbm_testing,testing,2025,169,counts_bbbm_tests,bbbm_testing_and_treatment,Female,0.0,Japan
...,...,...,...,...,...,...,...,...,...,...,...,...
683083,95_plus,/mnt/team/simulation_science/pub/models/vivari...,positive,bbbm_testing,testing,2100,446,counts_bbbm_tests,baseline,Male,0.0,Brazil
683084,95_plus,/mnt/team/simulation_science/pub/models/vivari...,positive,bbbm_testing,testing,2100,446,counts_bbbm_tests,bbbm_testing,Female,0.0,Brazil
683085,95_plus,/mnt/team/simulation_science/pub/models/vivari...,positive,bbbm_testing,testing,2100,446,counts_bbbm_tests,bbbm_testing,Male,0.0,Brazil
683086,95_plus,/mnt/team/simulation_science/pub/models/vivari...,positive,bbbm_testing,testing,2100,446,counts_bbbm_tests,bbbm_testing_and_treatment,Female,0.0,Brazil


In [106]:
bbbm_tests.dtypes

age_group            category
artifact_path        category
bbbm_test_results    category
entity               category
entity_type          category
event_year              int16
input_draw              int16
measure              category
scenario             category
sex                  category
value                 float64
location             category
dtype: object

In [110]:
bbbm_tests.astype({'age_group': 'str'}).query("age_group > '70_to_74' and value>0")

,age_group,artifact_path,bbbm_test_results,entity,entity_type,event_year,input_draw,measure,scenario,sex,value,location
273692,75_to_79,/mnt/team/simulation_science/pub/models/vivari...,positive,bbbm_testing,testing,2030,169,counts_bbbm_tests,bbbm_testing,Female,27253.0,Japan
273693,75_to_79,/mnt/team/simulation_science/pub/models/vivari...,positive,bbbm_testing,testing,2030,169,counts_bbbm_tests,bbbm_testing,Male,12051.0,Japan
273694,75_to_79,/mnt/team/simulation_science/pub/models/vivari...,positive,bbbm_testing,testing,2030,169,counts_bbbm_tests,bbbm_testing_and_treatment,Female,27253.0,Japan
273695,75_to_79,/mnt/team/simulation_science/pub/models/vivari...,positive,bbbm_testing,testing,2030,169,counts_bbbm_tests,bbbm_testing_and_treatment,Male,12051.0,Japan
273698,75_to_79,/mnt/team/simulation_science/pub/models/vivari...,positive,bbbm_testing,testing,2030,323,counts_bbbm_tests,bbbm_testing,Female,24494.0,Japan
...,...,...,...,...,...,...,...,...,...,...,...,...
611033,75_to_79,/mnt/team/simulation_science/pub/models/vivari...,negative,bbbm_testing,testing,2100,258,counts_bbbm_tests,bbbm_testing_and_treatment,Male,1340.0,Brazil
611036,75_to_79,/mnt/team/simulation_science/pub/models/vivari...,negative,bbbm_testing,testing,2100,446,counts_bbbm_tests,bbbm_testing,Female,2445.0,Brazil
611037,75_to_79,/mnt/team/simulation_science/pub/models/vivari...,negative,bbbm_testing,testing,2100,446,counts_bbbm_tests,bbbm_testing,Male,1218.0,Brazil
611038,75_to_79,/mnt/team/simulation_science/pub/models/vivari...,negative,bbbm_testing,testing,2100,446,counts_bbbm_tests,bbbm_testing_and_treatment,Female,2445.0,Brazil


In [111]:
bbbm_tests.age_group.unique()

['25_to_29', '30_to_34', '35_to_39', '40_to_44', '45_to_49', ..., '75_to_79', '80_to_84', '85_to_89', '90_to_94', '95_plus']
Length: 15
Categories (15, object): ['25_to_29' < '30_to_34' < '35_to_39' < '40_to_44' ... '80_to_84' < '85_to_89' < '90_to_94' < '95_plus']

In [112]:
bbbm_tests.query("value>0").age_group.unique()

['60_to_64', '65_to_69', '70_to_74', '75_to_79']
Categories (15, object): ['25_to_29' < '30_to_34' < '35_to_39' < '40_to_44' ... '80_to_84' < '85_to_89' < '90_to_94' < '95_plus']

In [115]:
bbbm_tests_prepped = results.process_bbbm_tests(bbbm_tests, mslt_results_prepped)
bbbm_tests_prepped

,age_group,event_year,input_draw,location,measure,scenario,sex,value,disease_stage,metric
0,60_to_64,2025,169,Japan,BBBM Tests,baseline,Female,0.000000,Preclinical AD,Number
1,60_to_64,2025,169,Japan,BBBM Tests,baseline,Male,0.000000,Preclinical AD,Number
2,60_to_64,2025,169,Japan,BBBM Tests,bbbm_testing,Female,0.000000,Preclinical AD,Number
3,60_to_64,2025,169,Japan,BBBM Tests,bbbm_testing,Male,0.000000,Preclinical AD,Number
4,60_to_64,2025,169,Japan,BBBM Tests,bbbm_testing_and_treatment,Female,0.000000,Preclinical AD,Number
...,...,...,...,...,...,...,...,...,...,...
303995,75_to_79,2100,169,United States of America,BBBM Positive Tests,BBBM Testing and Treatment,Male,141981.696594,Susceptible,Number
303996,75_to_79,2100,258,United States of America,BBBM Positive Tests,BBBM Testing and Treatment,Male,135267.959115,Susceptible,Number
303997,75_to_79,2100,323,United States of America,BBBM Positive Tests,BBBM Testing and Treatment,Male,129383.841038,Susceptible,Number
303998,75_to_79,2100,446,United States of America,BBBM Positive Tests,BBBM Testing and Treatment,Male,134673.657679,Susceptible,Number


In [120]:
bbbm_test_rate = results.calculate_rate(bbbm_tests_prepped)
bbbm_test_rate

,age_group,event_year,input_draw,location,sex,disease_stage,measure,metric,scenario,value
0,60_to_64,2025,169,Brazil,Female,Preclinical AD,BBBM Tests,Rate,baseline,0.000000
1,60_to_64,2025,169,Brazil,Female,Preclinical AD,BBBM Tests,Rate,bbbm_testing,0.000000
2,60_to_64,2025,169,Brazil,Female,Preclinical AD,BBBM Tests,Rate,bbbm_testing_and_treatment,0.000000
3,60_to_64,2025,169,Brazil,Female,Preclinical AD,Positive BBBM Tests,Rate,baseline,0.000000
4,60_to_64,2025,169,Brazil,Female,Preclinical AD,Positive BBBM Tests,Rate,bbbm_testing,0.000000
...,...,...,...,...,...,...,...,...,...,...
303995,75_to_79,2100,457,United States of America,Male,Preclinical AD,Positive BBBM Tests,Rate,bbbm_testing_and_treatment,0.002588
303996,75_to_79,2100,457,United States of America,Male,Susceptible,BBBM Tests,Rate,BBBM Testing Only,0.206670
303997,75_to_79,2100,457,United States of America,Male,Susceptible,BBBM Positive Tests,Rate,BBBM Testing Only,0.020667
303998,75_to_79,2100,457,United States of America,Male,Susceptible,BBBM Tests,Rate,BBBM Testing and Treatment,0.206670


In [104]:
bbbm_test_rate.query("value>0")

,age_group,event_year,input_draw,location,sex,disease_stage,measure,metric,scenario,value
324201,60_to_64,2030,169,Brazil,Female,Preclinical AD,BBBM Tests,Number,bbbm_testing,0.001475
324202,60_to_64,2030,169,Brazil,Female,Preclinical AD,BBBM Tests,Number,bbbm_testing_and_treatment,0.001475
324204,60_to_64,2030,169,Brazil,Female,Preclinical AD,Positive BBBM Tests,Number,bbbm_testing,0.001318
324205,60_to_64,2030,169,Brazil,Female,Preclinical AD,Positive BBBM Tests,Number,bbbm_testing_and_treatment,0.001318
324206,60_to_64,2030,169,Brazil,Female,Susceptible,BBBM Tests,Number,BBBM Testing Only,0.094602
...,...,...,...,...,...,...,...,...,...,...
623195,75_to_79,2100,457,United States of America,Male,Preclinical AD,Positive BBBM Tests,Number,bbbm_testing_and_treatment,0.002588
623196,75_to_79,2100,457,United States of America,Male,Susceptible,BBBM Tests,Number,BBBM Testing Only,0.206670
623197,75_to_79,2100,457,United States of America,Male,Susceptible,BBBM Positive Tests,Number,BBBM Testing Only,0.020667
623198,75_to_79,2100,457,United States of America,Male,Susceptible,BBBM Tests,Number,BBBM Testing and Treatment,0.206670


# CSF and PET Test Counts

In [125]:
csf_pet_filters = [('testing_state', 'in', ['csf', 'pet'])]
with Timer():
    csf_pet_tests = results.load_measure_from_batch_runs(
        'counts_baseline_tests_among_eligible', False, filters=csf_pet_filters
    )
print(len(csf_pet_tests), 'rows')
csf_pet_tests.head()

[('event_year', '>=', '2025'), ('testing_state', 'in', ['csf', 'pet'])]
{'all': PosixPath('/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/results/model8.7/model_spec/2025_11_05_15_36_29/results')}
{'United States of America': '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model8.3/united_states_of_america.hdf', 'Brazil': '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model8.3/brazil.hdf', 'China': '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model8.3/china.hdf', 'Germany': '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model8.3/germany.hdf', 'Israel': '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model8.3/israel.hdf', 'Japan': '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model8.3/japan.hdf', 'Spain': '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model8.3/spain.hdf', 'Swe

,age_group,artifact_path,entity,entity_type,event_year,input_draw,measure,scenario,sex,testing_state,value,location
0,25_to_29,/mnt/team/simulation_science/pub/models/vivari...,baseline_testing,testing,2025,169,counts_baseline_tests_among_eligible,baseline,Female,csf,0.0,Japan
1,25_to_29,/mnt/team/simulation_science/pub/models/vivari...,baseline_testing,testing,2025,169,counts_baseline_tests_among_eligible,baseline,Female,pet,0.0,Japan
2,25_to_29,/mnt/team/simulation_science/pub/models/vivari...,baseline_testing,testing,2025,169,counts_baseline_tests_among_eligible,baseline,Male,csf,0.0,Japan
3,25_to_29,/mnt/team/simulation_science/pub/models/vivari...,baseline_testing,testing,2025,169,counts_baseline_tests_among_eligible,baseline,Male,pet,0.0,Japan
4,25_to_29,/mnt/team/simulation_science/pub/models/vivari...,baseline_testing,testing,2025,169,counts_baseline_tests_among_eligible,bbbm_testing,Female,csf,0.0,Japan


In [126]:
csf_pet_tests

,age_group,artifact_path,entity,entity_type,event_year,input_draw,measure,scenario,sex,testing_state,value,location
0,25_to_29,/mnt/team/simulation_science/pub/models/vivari...,baseline_testing,testing,2025,169,counts_baseline_tests_among_eligible,baseline,Female,csf,0.0,Japan
1,25_to_29,/mnt/team/simulation_science/pub/models/vivari...,baseline_testing,testing,2025,169,counts_baseline_tests_among_eligible,baseline,Female,pet,0.0,Japan
2,25_to_29,/mnt/team/simulation_science/pub/models/vivari...,baseline_testing,testing,2025,169,counts_baseline_tests_among_eligible,baseline,Male,csf,0.0,Japan
3,25_to_29,/mnt/team/simulation_science/pub/models/vivari...,baseline_testing,testing,2025,169,counts_baseline_tests_among_eligible,baseline,Male,pet,0.0,Japan
4,25_to_29,/mnt/team/simulation_science/pub/models/vivari...,baseline_testing,testing,2025,169,counts_baseline_tests_among_eligible,bbbm_testing,Female,csf,0.0,Japan
...,...,...,...,...,...,...,...,...,...,...,...,...
683995,95_plus,/mnt/team/simulation_science/pub/models/vivari...,baseline_testing,testing,2100,446,counts_baseline_tests_among_eligible,bbbm_testing,Male,pet,419.0,Brazil
683996,95_plus,/mnt/team/simulation_science/pub/models/vivari...,baseline_testing,testing,2100,446,counts_baseline_tests_among_eligible,bbbm_testing_and_treatment,Female,csf,987.0,Brazil
683997,95_plus,/mnt/team/simulation_science/pub/models/vivari...,baseline_testing,testing,2100,446,counts_baseline_tests_among_eligible,bbbm_testing_and_treatment,Female,pet,775.0,Brazil
683998,95_plus,/mnt/team/simulation_science/pub/models/vivari...,baseline_testing,testing,2100,446,counts_baseline_tests_among_eligible,bbbm_testing_and_treatment,Male,csf,532.0,Brazil


In [128]:
csf_pet_tests.query("value>0").age_group.unique()

['30_to_34', '35_to_39', '40_to_44', '45_to_49', '50_to_54', ..., '75_to_79', '80_to_84', '85_to_89', '90_to_94', '95_plus']
Length: 14
Categories (15, object): ['25_to_29' < '30_to_34' < '35_to_39' < '40_to_44' ... '80_to_84' < '85_to_89' < '90_to_94' < '95_plus']

In [130]:
csf_pet_tests_prepped = results.process_csf_pet_tests(csf_pet_tests)
csf_pet_tests_prepped

19.842122 MB measure
6.618122 MB minuend
13.230122 MB subtrahend
4.341673 MB minuend re-indexed
8.673673 MB subtrahend re-indexed
8.673733 MB difference
15.054254 MB difference with reset index
15.510579 MB final difference


,age_group,artifact_path,entity,entity_type,event_year,input_draw,measure,scenario,sex,testing_state,value,location,metric
0,25_to_29,/mnt/team/simulation_science/pub/models/vivari...,baseline_testing,testing,2025,169,CSF Tests,baseline,Female,csf,0.0,Japan,Number
1,25_to_29,/mnt/team/simulation_science/pub/models/vivari...,baseline_testing,testing,2025,169,PET Tests,baseline,Female,pet,0.0,Japan,Number
2,25_to_29,/mnt/team/simulation_science/pub/models/vivari...,baseline_testing,testing,2025,169,CSF Tests,baseline,Male,csf,0.0,Japan,Number
3,25_to_29,/mnt/team/simulation_science/pub/models/vivari...,baseline_testing,testing,2025,169,PET Tests,baseline,Male,pet,0.0,Japan,Number
4,25_to_29,/mnt/team/simulation_science/pub/models/vivari...,baseline_testing,testing,2025,169,CSF Tests,bbbm_testing,Female,csf,0.0,Japan,Number
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1139995,95_plus,/mnt/team/simulation_science/pub/models/vivari...,baseline_testing,testing,2100,446,Averted PET Tests,bbbm_testing,Male,pet,12.0,Brazil,Number
1139996,95_plus,/mnt/team/simulation_science/pub/models/vivari...,baseline_testing,testing,2100,446,Averted CSF Tests,bbbm_testing_and_treatment,Female,csf,28.0,Brazil,Number
1139997,95_plus,/mnt/team/simulation_science/pub/models/vivari...,baseline_testing,testing,2100,446,Averted PET Tests,bbbm_testing_and_treatment,Female,pet,35.0,Brazil,Number
1139998,95_plus,/mnt/team/simulation_science/pub/models/vivari...,baseline_testing,testing,2100,446,Averted CSF Tests,bbbm_testing_and_treatment,Male,csf,8.0,Brazil,Number


# Treatment Counts

In [133]:
treatments_filters = [
    ('sub_entity', 'in',
     ['susceptible_to_treatment_to_waiting_for_treatment',
      'waiting_for_treatment_to_full_effect_long',
      'waiting_for_treatment_to_full_effect_short']
    )
]

with Timer():
    treatments = results.load_measure_from_batch_runs(
        'transition_count_treatment', False, filters=treatments_filters
    )
print(len(treatments), 'rows')
treatments.head()

[('event_year', '>=', '2025'), ('sub_entity', 'in', ['susceptible_to_treatment_to_waiting_for_treatment', 'waiting_for_treatment_to_full_effect_long', 'waiting_for_treatment_to_full_effect_short'])]
{'all': PosixPath('/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/results/model8.7/model_spec/2025_11_05_15_36_29/results')}
{'United States of America': '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model8.3/united_states_of_america.hdf', 'Brazil': '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model8.3/brazil.hdf', 'China': '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model8.3/china.hdf', 'Germany': '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model8.3/germany.hdf', 'Israel': '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model8.3/israel.hdf', 'Japan': '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/mod

,age_group,artifact_path,entity,entity_type,event_year,input_draw,measure,scenario,sex,sub_entity,value,location
0,25_to_29,/mnt/team/simulation_science/pub/models/vivari...,treatment,cause,2025,169,transition_count,baseline,Female,waiting_for_treatment_to_full_effect_long,0.0,Japan
1,25_to_29,/mnt/team/simulation_science/pub/models/vivari...,treatment,cause,2025,169,transition_count,baseline,Female,waiting_for_treatment_to_full_effect_short,0.0,Japan
2,25_to_29,/mnt/team/simulation_science/pub/models/vivari...,treatment,cause,2025,169,transition_count,baseline,Female,susceptible_to_treatment_to_waiting_for_treatment,0.0,Japan
3,25_to_29,/mnt/team/simulation_science/pub/models/vivari...,treatment,cause,2025,169,transition_count,baseline,Male,waiting_for_treatment_to_full_effect_long,0.0,Japan
4,25_to_29,/mnt/team/simulation_science/pub/models/vivari...,treatment,cause,2025,169,transition_count,baseline,Male,waiting_for_treatment_to_full_effect_short,0.0,Japan


In [135]:
start_treatment = [
        'waiting_for_treatment_to_full_effect_long',
        'waiting_for_treatment_to_full_effect_short']
treatments.query("sub_entity in @start_treatment and value>0").age_group.unique()

['60_to_64', '65_to_69', '70_to_74', '75_to_79', '80_to_84']
Categories (15, object): ['25_to_29' < '30_to_34' < '35_to_39' < '40_to_44' ... '80_to_84' < '85_to_89' < '90_to_94' < '95_plus']

In [136]:
treatments.query("sub_entity=='susceptible_to_treatment_to_waiting_for_treatment' and value>0").age_group.unique()

['60_to_64', '65_to_69', '70_to_74', '75_to_79']
Categories (15, object): ['25_to_29' < '30_to_34' < '35_to_39' < '40_to_44' ... '80_to_84' < '85_to_89' < '90_to_94' < '95_plus']

In [144]:
treatments_prepped = results.process_treatments(treatments, mslt_results_prepped)
treatments_prepped

,age_group,event_year,input_draw,measure,scenario,sex,value,location,disease_stage
0,60_to_64,2025,169,Medication Completion,baseline,Female,0.000000,Japan,Preclinical AD
1,60_to_64,2025,169,Medication Discontinuation,baseline,Female,0.000000,Japan,Preclinical AD
2,60_to_64,2025,169,Medication Completion,baseline,Male,0.000000,Japan,Preclinical AD
3,60_to_64,2025,169,Medication Discontinuation,baseline,Male,0.000000,Japan,Preclinical AD
4,60_to_64,2025,169,Medication Completion,bbbm_testing,Female,0.000000,Japan,Preclinical AD
...,...,...,...,...,...,...,...,...,...
516795,75_to_79,2100,169,Medication Initiation,BBBM Testing and Treatment,Male,42594.508978,United States of America,Susceptible
516796,75_to_79,2100,258,Medication Initiation,BBBM Testing and Treatment,Male,40580.387735,United States of America,Susceptible
516797,75_to_79,2100,323,Medication Initiation,BBBM Testing and Treatment,Male,38815.152311,United States of America,Susceptible
516798,75_to_79,2100,446,Medication Initiation,BBBM Testing and Treatment,Male,40402.097304,United States of America,Susceptible


In [151]:
with Timer():
    treatments_output = results.summarize_and_beautify(treatments_prepped, 'disease_stage')

print(len(treatments_output), 'rows')
treatments_output.head()

2025-11-20 12:55:26.255038
2025-11-20 12:55:29.383393
2025-11-20 12:59:07.813785
Elapsed time: 0:03:41.569582
161120 rows


,Year,Location,Age,Sex,Disease Stage,Scenario,Measure,Metric,Mean,95% UI Lower,95% UI Upper
0,2025,Brazil,60_to_64,Female,Preclinical AD,Reference,Medication Completion,Number,0.0,0.0,0.0
1,2025,Brazil,60_to_64,Male,Preclinical AD,Reference,Medication Completion,Number,0.0,0.0,0.0
2,2025,Brazil,60_to_64,Female,Preclinical AD,BBBM Testing Only,Medication Completion,Number,0.0,0.0,0.0
3,2025,Brazil,60_to_64,Male,Preclinical AD,BBBM Testing Only,Medication Completion,Number,0.0,0.0,0.0
4,2025,Brazil,60_to_64,Female,Preclinical AD,BBBM Testing and Treatment,Medication Completion,Number,0.0,0.0,0.0


In [154]:
treatments_output

,Year,Location,Age,Sex,Disease Stage,Scenario,Measure,Metric,Mean,95% UI Lower,95% UI Upper
0,2025,Brazil,60_to_64,Female,Preclinical AD,Reference,Medication Completion,Number,0.000000,0.000000,0.000000
1,2025,Brazil,60_to_64,Male,Preclinical AD,Reference,Medication Completion,Number,0.000000,0.000000,0.000000
2,2025,Brazil,60_to_64,Female,Preclinical AD,BBBM Testing Only,Medication Completion,Number,0.000000,0.000000,0.000000
3,2025,Brazil,60_to_64,Male,Preclinical AD,BBBM Testing Only,Medication Completion,Number,0.000000,0.000000,0.000000
4,2025,Brazil,60_to_64,Female,Preclinical AD,BBBM Testing and Treatment,Medication Completion,Number,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...
161115,2100,United States of America,80_to_84,Male,Preclinical AD,Reference,Medication Initiation,Rate,0.000000,0.000000,0.000000
161116,2100,United States of America,80_to_84,Female,Preclinical AD,BBBM Testing Only,Medication Initiation,Rate,0.000000,0.000000,0.000000
161117,2100,United States of America,80_to_84,Male,Preclinical AD,BBBM Testing Only,Medication Initiation,Rate,0.000000,0.000000,0.000000
161118,2100,United States of America,80_to_84,Female,Preclinical AD,BBBM Testing and Treatment,Medication Initiation,Rate,0.000132,0.000019,0.000355


In [156]:
treatments_output.query("`Disease Stage` == 'Preclinical AD'").Year.unique()

[2025, 2026, 2027, 2028, 2029, ..., 2096, 2097, 2098, 2099, 2100]
Length: 76
Categories (76, int64): [2025, 2026, 2027, 2028, ..., 2097, 2098, 2099, 2100]

In [145]:
treatments_prepped.age_group.unique()

['60_to_64', '65_to_69', '70_to_74', '75_to_79', '80_to_84']
Categories (5, object): ['60_to_64', '65_to_69', '70_to_74', '75_to_79', '80_to_84']

In [147]:
treatment_rates = results.calculate_rate(treatments_prepped)
treatment_rates

,age_group,event_year,input_draw,location,sex,disease_stage,measure,scenario,value,metric
0,60_to_64,2025,169,Brazil,Female,Preclinical AD,Medication Completion,baseline,0.000000,Rate
1,60_to_64,2025,169,Brazil,Female,Preclinical AD,Medication Discontinuation,baseline,0.000000,Rate
2,60_to_64,2025,169,Brazil,Female,Preclinical AD,Medication Completion,bbbm_testing,0.000000,Rate
3,60_to_64,2025,169,Brazil,Female,Preclinical AD,Medication Discontinuation,bbbm_testing,0.000000,Rate
4,60_to_64,2025,169,Brazil,Female,Preclinical AD,Medication Completion,bbbm_testing_and_treatment,0.000000,Rate
...,...,...,...,...,...,...,...,...,...,...
516795,80_to_84,2100,457,United States of America,Male,Preclinical AD,Medication Initiation,baseline,0.000000,Rate
516796,80_to_84,2100,457,United States of America,Male,Preclinical AD,Medication Initiation,bbbm_testing,0.000000,Rate
516797,80_to_84,2100,457,United States of America,Male,Preclinical AD,Medication Initiation,bbbm_testing,0.000000,Rate
516798,80_to_84,2100,457,United States of America,Male,Preclinical AD,Medication Initiation,bbbm_testing_and_treatment,0.000090,Rate


In [149]:
treatment_rates.query("disease_stage=='Susceptible'")

,age_group,event_year,input_draw,location,sex,disease_stage,measure,scenario,value,metric
12,60_to_64,2025,169,Brazil,Female,Susceptible,Medication Initiation,BBBM Testing Only,0.000000,Rate
13,60_to_64,2025,169,Brazil,Female,Susceptible,Medication Initiation,BBBM Testing and Treatment,0.000000,Rate
26,60_to_64,2025,169,Brazil,Male,Susceptible,Medication Initiation,BBBM Testing Only,0.000000,Rate
27,60_to_64,2025,169,Brazil,Male,Susceptible,Medication Initiation,BBBM Testing and Treatment,0.000000,Rate
40,60_to_64,2025,169,China,Female,Susceptible,Medication Initiation,BBBM Testing Only,0.000000,Rate
...,...,...,...,...,...,...,...,...,...,...
425571,75_to_79,2100,457,United Kingdom,Male,Susceptible,Medication Initiation,BBBM Testing and Treatment,0.012777,Rate
425584,75_to_79,2100,457,United States of America,Female,Susceptible,Medication Initiation,BBBM Testing Only,0.000000,Rate
425585,75_to_79,2100,457,United States of America,Female,Susceptible,Medication Initiation,BBBM Testing and Treatment,0.005719,Rate
425598,75_to_79,2100,457,United States of America,Male,Susceptible,Medication Initiation,BBBM Testing Only,0.000000,Rate


In [141]:
mslt_results_prepped.query("measure.str.contains('Medication')")

,event_year,location,age_group,sex,disease_stage,scenario,measure,metric,input_draw,value
3,2025,Brazil,60_to_64,Female,Susceptible,BBBM Testing Only,Medication Initiation,Number,169,0.000000
8,2025,Brazil,60_to_64,Female,Susceptible,BBBM Testing Only,Medication Initiation,Number,258,0.000000
13,2025,Brazil,60_to_64,Female,Susceptible,BBBM Testing Only,Medication Initiation,Number,323,0.000000
18,2025,Brazil,60_to_64,Female,Susceptible,BBBM Testing Only,Medication Initiation,Number,446,0.000000
19,2025,Brazil,60_to_64,Female,Susceptible,BBBM Testing Only,Medication Initiation,Number,457,0.000000
...,...,...,...,...,...,...,...,...,...,...
151978,2100,United States of America,75_to_79,Male,Susceptible,BBBM Testing and Treatment,Medication Initiation,Number,169,42594.508978
151983,2100,United States of America,75_to_79,Male,Susceptible,BBBM Testing and Treatment,Medication Initiation,Number,258,40580.387735
151988,2100,United States of America,75_to_79,Male,Susceptible,BBBM Testing and Treatment,Medication Initiation,Number,323,38815.152311
151993,2100,United States of America,75_to_79,Male,Susceptible,BBBM Testing and Treatment,Medication Initiation,Number,446,40402.097304


In [142]:
mslt_results_prepped.measure.unique()

['BBBM Tests', 'BBBM Positive Tests', 'Medication Initiation']
Categories (3, object): ['BBBM Positive Tests', 'BBBM Tests', 'Medication Initiation']

# Scratchwork

In [69]:
bbbm_tests_prepped

,age_group,artifact_path,entity,entity_type,event_year,input_draw,location,measure,scenario,sex,value,disease_stage,metric
0,25_to_29,/mnt/team/simulation_science/pub/models/vivari...,bbbm_testing,testing,2025,169,Japan,BBBM Tests,baseline,Female,0.0,Preclinical AD,Number
1,25_to_29,/mnt/team/simulation_science/pub/models/vivari...,bbbm_testing,testing,2025,169,Japan,BBBM Tests,baseline,Male,0.0,Preclinical AD,Number
2,25_to_29,/mnt/team/simulation_science/pub/models/vivari...,bbbm_testing,testing,2025,169,Japan,BBBM Tests,bbbm_testing,Female,0.0,Preclinical AD,Number
3,25_to_29,/mnt/team/simulation_science/pub/models/vivari...,bbbm_testing,testing,2025,169,Japan,BBBM Tests,bbbm_testing,Male,0.0,Preclinical AD,Number
4,25_to_29,/mnt/team/simulation_science/pub/models/vivari...,bbbm_testing,testing,2025,169,Japan,BBBM Tests,bbbm_testing_and_treatment,Female,0.0,Preclinical AD,Number
...,...,...,...,...,...,...,...,...,...,...,...,...,...
683995,95_plus,/mnt/team/simulation_science/pub/models/vivari...,bbbm_testing,testing,2100,446,Brazil,Positive BBBM Tests,baseline,Male,0.0,Preclinical AD,Number
683996,95_plus,/mnt/team/simulation_science/pub/models/vivari...,bbbm_testing,testing,2100,446,Brazil,Positive BBBM Tests,bbbm_testing,Female,0.0,Preclinical AD,Number
683997,95_plus,/mnt/team/simulation_science/pub/models/vivari...,bbbm_testing,testing,2100,446,Brazil,Positive BBBM Tests,bbbm_testing,Male,0.0,Preclinical AD,Number
683998,95_plus,/mnt/team/simulation_science/pub/models/vivari...,bbbm_testing,testing,2100,446,Brazil,Positive BBBM Tests,bbbm_testing_and_treatment,Female,0.0,Preclinical AD,Number


In [66]:
column_name_map = {
            'event_year': 'Year',
            'age_group': 'Age',
            'location': 'Location',
            'sex': 'Sex',
            'scenario': 'Scenario',
            'measure': 'Measure',
            'metric': 'Metric',
            'disease_stage': 'Disease Stage',
            'input_draw': 'Draw',
            'value': 'Value',
            'mean': 'Mean',
            'lower': '95% UI Lower',
            'upper': '95% UI Upper',
        }
reverse_map = {v: k for k, v in column_name_map.items()}
reverse_map

{'Year': 'event_year',
 'Age': 'age_group',
 'Location': 'location',
 'Sex': 'sex',
 'Scenario': 'scenario',
 'Measure': 'measure',
 'Metric': 'metric',
 'Disease Stage': 'disease_stage',
 'Draw': 'input_draw',
 'Value': 'value',
 'Mean': 'mean',
 '95% UI Lower': 'lower',
 '95% UI Upper': 'upper'}

In [68]:
temp = mslt_results.rename(columns=reverse_map)
temp

,event_year,location,age_group,sex,disease_stage,scenario,measure,metric,input_draw,value
0,2025,Brazil,60_to_64,Female,Susceptible,BBBM Testing Only,BBBM Tests,Number,113,0.000000
1,2025,Brazil,60_to_64,Female,Susceptible,BBBM Testing Only,BBBM Tests,Number,13,0.000000
2,2025,Brazil,60_to_64,Female,Susceptible,BBBM Testing Only,BBBM Tests,Number,158,0.000000
3,2025,Brazil,60_to_64,Female,Susceptible,BBBM Testing Only,BBBM Tests,Number,169,0.000000
4,2025,Brazil,60_to_64,Female,Susceptible,BBBM Testing Only,BBBM Tests,Number,177,0.000000
...,...,...,...,...,...,...,...,...,...,...
151995,2100,United States of America,75_to_79,Male,Susceptible,BBBM Testing and Treatment,Improper Medication Uses,Number,46,41101.303607
151996,2100,United States of America,75_to_79,Male,Susceptible,BBBM Testing and Treatment,Improper Medication Uses,Number,460,41691.942867
151997,2100,United States of America,75_to_79,Male,Susceptible,BBBM Testing and Treatment,Improper Medication Uses,Number,480,39195.330257
151998,2100,United States of America,75_to_79,Male,Susceptible,BBBM Testing and Treatment,Improper Medication Uses,Number,499,40424.816911


In [70]:
pd.concat([bbbm_tests_prepped, temp])

,age_group,artifact_path,entity,entity_type,event_year,input_draw,location,measure,scenario,sex,value,disease_stage,metric
0,25_to_29,/mnt/team/simulation_science/pub/models/vivari...,bbbm_testing,testing,2025,169,Japan,BBBM Tests,baseline,Female,0.000000,Preclinical AD,Number
1,25_to_29,/mnt/team/simulation_science/pub/models/vivari...,bbbm_testing,testing,2025,169,Japan,BBBM Tests,baseline,Male,0.000000,Preclinical AD,Number
2,25_to_29,/mnt/team/simulation_science/pub/models/vivari...,bbbm_testing,testing,2025,169,Japan,BBBM Tests,bbbm_testing,Female,0.000000,Preclinical AD,Number
3,25_to_29,/mnt/team/simulation_science/pub/models/vivari...,bbbm_testing,testing,2025,169,Japan,BBBM Tests,bbbm_testing,Male,0.000000,Preclinical AD,Number
4,25_to_29,/mnt/team/simulation_science/pub/models/vivari...,bbbm_testing,testing,2025,169,Japan,BBBM Tests,bbbm_testing_and_treatment,Female,0.000000,Preclinical AD,Number
...,...,...,...,...,...,...,...,...,...,...,...,...,...
151995,75_to_79,NaN,NaN,NaN,2100,46,United States of America,Improper Medication Uses,BBBM Testing and Treatment,Male,41101.303607,Susceptible,Number
151996,75_to_79,NaN,NaN,NaN,2100,460,United States of America,Improper Medication Uses,BBBM Testing and Treatment,Male,41691.942867,Susceptible,Number
151997,75_to_79,NaN,NaN,NaN,2100,480,United States of America,Improper Medication Uses,BBBM Testing and Treatment,Male,39195.330257,Susceptible,Number
151998,75_to_79,NaN,NaN,NaN,2100,499,United States of America,Improper Medication Uses,BBBM Testing and Treatment,Male,40424.816911,Susceptible,Number


In [77]:
temp.query("measure.str.contains('Medication') and value > 0 and scenario=='BBBM Testing Only'")

,event_year,location,age_group,sex,disease_stage,scenario,measure,metric,input_draw,value
1000,2030,Brazil,60_to_64,Female,Susceptible,BBBM Testing Only,Improper Medication Uses,Number,113,24115.316094
1001,2030,Brazil,60_to_64,Female,Susceptible,BBBM Testing Only,Improper Medication Uses,Number,13,25432.967559
1002,2030,Brazil,60_to_64,Female,Susceptible,BBBM Testing Only,Improper Medication Uses,Number,158,24846.791963
1003,2030,Brazil,60_to_64,Female,Susceptible,BBBM Testing Only,Improper Medication Uses,Number,169,24018.184411
1004,2030,Brazil,60_to_64,Female,Susceptible,BBBM Testing Only,Improper Medication Uses,Number,177,21771.561248
...,...,...,...,...,...,...,...,...,...,...
151995,2100,United States of America,75_to_79,Male,Susceptible,BBBM Testing Only,Improper Medication Uses,Number,46,41101.303607
151996,2100,United States of America,75_to_79,Male,Susceptible,BBBM Testing Only,Improper Medication Uses,Number,460,41691.942867
151997,2100,United States of America,75_to_79,Male,Susceptible,BBBM Testing Only,Improper Medication Uses,Number,480,39195.330257
151998,2100,United States of America,75_to_79,Male,Susceptible,BBBM Testing Only,Improper Medication Uses,Number,499,40424.816911


In [78]:
temp.measure.unique()

array(['BBBM Tests', 'BBBM False Positive Tests',
       'Improper Medication Uses'], dtype=object)

In [83]:
temp.replace(
    {'measure': {'BBBM False Positive Tests': 'BBBM Positive Tests',
 'Improper Medication Uses': 'Medication Initiation'}})


,event_year,location,age_group,sex,disease_stage,scenario,measure,metric,input_draw,value
0,2025,Brazil,60_to_64,Female,Susceptible,BBBM Testing Only,BBBM Tests,Number,113,0.000000
1,2025,Brazil,60_to_64,Female,Susceptible,BBBM Testing Only,BBBM Tests,Number,13,0.000000
2,2025,Brazil,60_to_64,Female,Susceptible,BBBM Testing Only,BBBM Tests,Number,158,0.000000
3,2025,Brazil,60_to_64,Female,Susceptible,BBBM Testing Only,BBBM Tests,Number,169,0.000000
4,2025,Brazil,60_to_64,Female,Susceptible,BBBM Testing Only,BBBM Tests,Number,177,0.000000
...,...,...,...,...,...,...,...,...,...,...
151995,2100,United States of America,75_to_79,Male,Susceptible,BBBM Testing and Treatment,Medication Initiation,Number,46,41101.303607
151996,2100,United States of America,75_to_79,Male,Susceptible,BBBM Testing and Treatment,Medication Initiation,Number,460,41691.942867
151997,2100,United States of America,75_to_79,Male,Susceptible,BBBM Testing and Treatment,Medication Initiation,Number,480,39195.330257
151998,2100,United States of America,75_to_79,Male,Susceptible,BBBM Testing and Treatment,Medication Initiation,Number,499,40424.816911


In [82]:
bbbm_tests_prepped.dtypes

age_group        category
artifact_path    category
entity           category
entity_type      category
event_year       category
input_draw       category
location         category
measure          category
scenario         category
sex              category
value             float64
disease_stage    category
metric           category
dtype: object

In [ ]:
testing = temp.query("scenario=='BBBM Testing Only'").drop(columns='scenario')
treatment = temp.query("scenario=='BBBM Testing and Treatment'").drop(columns='scenario')
# This shows that the whole dataframe was copied to the testing-only
# scenario, including medication initiations, which is incorrect
results.ops.compare_values(testing, treatment)

age_group,disease_stage,event_year,input_draw,location,measure,metric,sex


In [92]:
mslt_results_processed = results.process_mslt_results(mslt_results)
mslt_results_processed

,event_year,location,age_group,sex,disease_stage,scenario,measure,metric,input_draw,value
3,2025,Brazil,60_to_64,Female,Susceptible,BBBM Testing Only,BBBM Tests,Number,169,0.000000
8,2025,Brazil,60_to_64,Female,Susceptible,BBBM Testing Only,BBBM Tests,Number,258,0.000000
13,2025,Brazil,60_to_64,Female,Susceptible,BBBM Testing Only,BBBM Tests,Number,323,0.000000
18,2025,Brazil,60_to_64,Female,Susceptible,BBBM Testing Only,BBBM Tests,Number,446,0.000000
19,2025,Brazil,60_to_64,Female,Susceptible,BBBM Testing Only,BBBM Tests,Number,457,0.000000
...,...,...,...,...,...,...,...,...,...,...
151978,2100,United States of America,75_to_79,Male,Susceptible,BBBM Testing and Treatment,Medication Initiation,Number,169,42594.508978
151983,2100,United States of America,75_to_79,Male,Susceptible,BBBM Testing and Treatment,Medication Initiation,Number,258,40580.387735
151988,2100,United States of America,75_to_79,Male,Susceptible,BBBM Testing and Treatment,Medication Initiation,Number,323,38815.152311
151993,2100,United States of America,75_to_79,Male,Susceptible,BBBM Testing and Treatment,Medication Initiation,Number,446,40402.097304


In [93]:
mslt_results_processed.input_draw.unique()

[169, 258, 323, 446, 457]
Categories (5, int64): [169, 258, 323, 446, 457]

In [94]:
print_memory_usage(mslt_results_processed)

4.565503 MB 


In [96]:
mslt_results_processed.measure.unique()

['BBBM Tests', 'BBBM Positive Tests', 'Medication Initiation']
Categories (3, object): ['BBBM Positive Tests', 'BBBM Tests', 'Medication Initiation']